# Concat of all DEBIAS M SCRIPTS FOR TESTING

## Torch Script: base only

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression
import pytorch_lightning as pl
import torch
from pytorch_lightning import LightningModule, Trainer, seed_everything
from torch import Tensor, nn
from torch.nn import functional as F
from torch.nn.functional import softmax, pairwise_distance
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torchmetrics.functional import accuracy

from pl_bolts.datamodules import SklearnDataModule

from argparse import ArgumentParser
from typing import Any, Dict, List, Tuple, Type
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

ImportError: Matplotlib requires dateutil>=2.7; you have 2.2

In [ ]:
def rescale(xx):
    return(xx/xx.sum(axis=1)[:, np.newaxis] )

In [ ]:
class PL_DEBIASM(pl.LightningModule):
    """Logistic regression model."""

    def __init__(
        self,
        X, 
        batch_sim_strength: float,
        input_dim: int,
        num_classes: int,
        bias: bool = True,
        learning_rate: float = 1e-4,
        optimizer: Type[Optimizer] = Adam,
        l1_strength: float = 0.0,
        l2_strength: float = 0.0,
        w_l2 : float = 0.0,
        use_log: bool=False,
        **kwargs: Any,
    ) -> None:
        """
        Args:
            input_dim: number of dimensions of the input (at least 1)
            num_classes: number of class labels (binary: 2, multi-class: >2)
            bias: specifies if a constant or intercept should be fitted (equivalent to fit_intercept in sklearn)
            learning_rate: learning_rate for the optimizer
            optimizer: the optimizer to use (default: ``Adam``)
            l1_strength: L1 regularization strength (default: ``0.0``)
            l2_strength: L2 regularization strength (default: ``0.0``)
        """
        super().__init__()
        self.save_hyperparameters() # save hyperparams upon init
        self.optimizer = optimizer # Adam optimizer
        
        # main model component: check hparams for input
        self.linear = nn.Linear(in_features=self.hparams.input_dim, 
                                out_features=self.hparams.num_classes, 
                                bias=bias) 

        self.X=X[:, 1:] # all cols except batch number
        self.bs=X[:, 0].long() # batch number in long datatype
        self.unique_bs=self.bs.unique().long() # set of batch values (starts at 0)
        self.n_batches=self.unique_bs.max()+1 # number of unique batches
        self.batch_weights = torch.nn.Parameter(data = torch.zeros(self.n_batches,
                                                                   input_dim)) # init batch weights as 0's (not 1's?)

        self.batch_sim_str=batch_sim_strength # this is the batch similarity metric
        
        
    def forward(self, x: Tensor) -> Tensor:
        batch_inds, x = x[:, 0], x[:, 1:]

        # x gets normalized by batch weights here! Can I just return this instead?
        # ^^^ No! This seems to skip learning entirely.
        x = F.normalize( torch.pow(2, self.batch_weights[batch_inds.long()] ) * x, p=1 )
        x = self.linear(x)
        y_hat = softmax(x)
        return y_hat

    
    def training_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        x, y = batch
        
        # flatten any input
        x = x.view(x.size(0), -1)

        y_hat = self.forward(x)
        
        
        # PyTorch cross_entropy function combines log_softmax and nll_loss in single function
        loss = F.cross_entropy(y_hat, y, reduction="sum")
        
        # L1 regularizer
        if self.hparams.l1_strength > 0:
            l1_reg = self.linear.weight.abs().sum()
            loss += self.hparams.l1_strength * l1_reg

        # L2 regularizer
        if self.hparams.l2_strength > 0:
            l2_reg = self.linear.weight.pow(2).sum()
            loss += self.hparams.l2_strength * l2_reg
            
        # L2 regularizer
        if self.hparams.l2_strength > 0:
            l2_reg = self.linear.weight.pow(2).sum()
            loss += self.hparams.l2_strength * l2_reg
            
        # DOMAIN similarity regularizer / bias correction
        if self.batch_sim_str > 0:
            x1 = torch.stack( [ ( torch.pow(2, self.batch_weights\
                                          )[torch.where(self.unique_bs==a)[0]] * \
                    (self.X[ torch.where(self.bs==a)[0] ] )  \
                   ).mean(axis=0) for a in self.unique_bs ] )

            x1 = F.normalize(x1, p=1)

            loss += sum( [pairwise_distance(x1, a) for a in x1] ).sum() *\
                                    self.batch_sim_str
            
        # L2 regularizer for bias weight    
        if self.hparams.w_l2 > 0:
            # L2 regularizer for weighting parameter
            l2_reg = self.batch_weights.pow(2).sum()
            loss += self.hparams.w_l2 * l2_reg



        loss /= float( x.size(0) )
        return {"loss": loss}

    def validation_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.forward(x)
         # PyTorch cross_entropy function combines log_softmax and nll_loss in single function
        loss = F.cross_entropy(y_hat, y, reduction="sum")
        
        # L1 regularizer
        if self.hparams.l1_strength > 0:
            l1_reg = self.linear.weight.abs().sum()
            loss += self.hparams.l1_strength * l1_reg

        # L2 regularizer
        if self.hparams.l2_strength > 0:
            l2_reg = self.linear.weight.pow(2).sum()
            loss += self.hparams.l2_strength * l2_reg
        
        self.log('val_loss', loss)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        acc = 0
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        
        return {"val_loss": val_loss}

    def test_step(self, batch: Tuple[Tensor, Tensor], batch_idx: int) -> Dict[str, Tensor]:
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.forward(x)
        acc = 0#
        return {"test_loss": F.cross_entropy(y_hat, y),
                "acc": acc}

    def test_epoch_end(self, outputs: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        acc = 0
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        tensorboard_logs = {"test_ce_loss": test_loss, "test_acc": acc}
        progress_bar_metrics = tensorboard_logs
        return {"test_loss": test_loss,
                "log": tensorboard_logs,
                "progress_bar": progress_bar_metrics}

    def configure_optimizers(self) -> Optimizer:
        return self.optimizer(self.parameters(), lr=self.hparams.learning_rate)

    @staticmethod
    def add_model_specific_args(parent_parser: ArgumentParser) -> ArgumentParser:
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument("--learning_rate", type=float, default=0.0001)
        parser.add_argument("--input_dim", type=int, default=None)
        parser.add_argument("--num_classes", type=int, default=None)
        parser.add_argument("--bias", default="store_true")
        parser.add_argument("--batch_size", type=int, default=100)
        
        return parser

In [5]:
def DEBIASM_train_and_pred(X_train, 
                         X_val, 
                         y_train, 
                         y_val,
                         batch_sim_strength=1,
                         w_l2 = 0,
                         batch_size=None,
                         learning_rate=0.005,
                         l2_strength=0,
                         includes_batches=False,
                         val_split=0.1, 
                         test_split=0,
                         min_epochs=15, 
#                          use_log=False, 
                         verbose=False
                         ):
    
    if batch_size is None:
        batch_size=X_train.shape[0]
    
    baseline_mod = LogisticRegression(max_iter=2500)
    baseline_mod.fit(rescale( X_train[:, 1:]), y_train)
        
        
    model = PL_DEBIASM( X = torch.tensor( np.vstack((X_train, X_val)) ),
                      batch_sim_strength = batch_sim_strength,
                      input_dim = X_train.shape[1]-1, 
                      num_classes = 2, 
                      batch_size = batch_size,
                      learning_rate = learning_rate,
                      l2_strength = l2_strength, 
                      w_l2 = w_l2
                    )
    
    ## initialize parameters to lbe similar to standard logistic regression
    model.linear.weight.data[0]=-torch.tensor( baseline_mod.coef_[0] )
    model.linear.weight.data[1]= torch.tensor( baseline_mod.coef_[0] )

    ## build pl dataloader
    dm = SklearnDataModule(X_train, 
                           y_train.astype(int),
                           val_split=val_split,
                           test_split=test_split
                           )

    ## run training
    trainer = pl.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=2)], 
                             check_val_every_n_epoch=2, 
                             weights_summary=None, 
                             progress_bar_refresh_rate=0,#verbose, 
                             min_epochs=min_epochs
                            )
    trainer.fit(model, 
                train_dataloaders=dm.train_dataloader(), 
                val_dataloaders=dm.val_dataloader()
               )
    
    ## get val predictions
    val_preds = model.forward( torch.tensor( X_val ).float() )[:, 1].detach().numpy()
    
    ## return predictions and the model
    return( val_preds, model )

## SKLEARN Script: base only

In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn.functional import pairwise_distance
from sklearn.base import BaseEstimator

In [7]:
def batch_weight_feature_and_nbatchpairs_scaling(strength, df_with_batch):
    nbs = df_with_batch.iloc[:, 0].nunique()
    w = nbs * (nbs - 1) / 2
    return(strength /( w * ( df_with_batch.shape[1] - 1 ) ) )

In [8]:
class DebiasMClassifier(BaseEstimator):
    """DebiasMClassifier: an sklean-style wrapper for the DEBIAS-M torch implementation."""
    def __init__(self,
                 *, 
                 batch_str = 'infer',
                 learning_rate=0.005, 
                 min_epochs=25,
                 l2_strength=0,
                 w_l2=0,
                 random_state=None,
                 x_val=0
                 ):
        
        self.learning_rate=learning_rate
        self.min_epochs=min_epochs
        self.l2_strength=l2_strength
        self.w_l2=w_l2
        self.batch_str=batch_str
        self.x_val = x_val
        self.random_state=random_state
        
    def fit(self, X, y, sample_weight=None):
        """Fit the baseline classifier.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        if self.batch_str=='infer':
            self.batch_str = batch_weight_feature_and_nbatchpairs_scaling(1e4, pd.DataFrame(X) )
            
        self.classes_ = np.unique(y)
        preds, mod = DEBIASM_train_and_pred(
                                            X, 
                                            self.x_val, 
                                            y, 
                                            0,
                                            batch_sim_strength = self.batch_str,
                                            learning_rate=self.learning_rate,
                                            min_epochs= self.min_epochs,
                                            l2_strength=self.l2_strength,
                                            w_l2 = self.w_l2
                                            )
        self.model = mod
        self.val_preds = preds
        
        return self

    def predict(self, X):
        """Perform classification on test vectors X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Test data.

        Returns
        -------
        y : array-like of shape (n_samples,) or (n_samples, n_outputs)
            Predicted target values for X.
        """
        y = self.model.forward( torch.tensor( X ).float() ).detach().numpy()[:, 1]>0.5
        return y

    def predict_proba(self, X):
        """
        Return probability estimates for the test vectors X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Test data.

        Returns
        -------
        P : ndarray of shape (n_samples, n_classes) or list of such arrays
            Returns the probability of the sample for each class in
            the model, where classes are ordered arithmetically, for each
            output.
        """
        
        P = self.model.forward( torch.tensor( X ).float() ).detach().numpy()
        
        return P

## Test script content

In [9]:
np.random.seed(123)
n_samples = 96*5
n_batches = 5
n_features = 100

## the read count matrix
X = ( np.random.rand(n_samples, n_features) * 1000 ).astype(int)

## the labels
y = np.random.rand(n_samples)>0.5

## the batches
batches = ( np.random.rand(n_samples) * n_batches ).astype(int)

X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch[:5, :5]

array([[  4, 696, 286, 226, 551],
       [  4, 513, 666, 105, 130],
       [  3, 542,  66, 653, 996],
       [  3,  16, 721,   7,  84],
       [  1, 456, 279, 932, 314]])

In [10]:
## set the valdiation batch to '4'
val_inds = batches==4
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
y_train, y_val = y[~val_inds], y[val_inds]

In [23]:
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training

In [19]:
dmc.w_l2 

# init to zero, single value
# still zero after fitting

0

In [24]:
dmc

DebiasMClassifier(x_val=array([[  4, 696, 286, ..., 398, 240, 343],
       [  4, 513, 666, ..., 744, 472, 121],
       [  4, 896, 223, ..., 347,   4, 294],
       ...,
       [  4, 648, 845, ..., 791, 976, 578],
       [  4, 318, 960, ..., 640, 879, 992],
       [  4, 618, 300, ..., 912, 996, 621]]))

In [26]:
dmc.model.batch_weights

Parameter containing:
tensor([[ 0.0439, -0.0139,  0.1708, -0.1313,  0.0761, -0.1367,  0.1163,  0.0613,
          0.1345,  0.1039,  0.1212,  0.0678, -0.0489, -0.0657,  0.1246,  0.0138,
          0.0152, -0.0360, -0.0909, -0.0733,  0.0362, -0.0414, -0.1912, -0.0985,
         -0.0687,  0.0607,  0.0506,  0.0653,  0.1270,  0.1072, -0.0203,  0.0625,
         -0.1134,  0.0399,  0.0888,  0.0363,  0.0021,  0.1288, -0.0716, -0.0016,
          0.0871,  0.1119,  0.0354, -0.0106, -0.1430,  0.0335, -0.0196,  0.0220,
          0.0846,  0.0781,  0.0415, -0.1406,  0.0696,  0.1458, -0.0699,  0.0733,
          0.1377, -0.0186,  0.1392,  0.0299,  0.0863,  0.0697,  0.0126,  0.0205,
          0.0285,  0.0820, -0.1087, -0.0371, -0.1158, -0.0785, -0.0591,  0.1200,
         -0.0731,  0.1671,  0.1106,  0.0124, -0.0850,  0.0512, -0.1413,  0.0956,
          0.0716, -0.1063, -0.0589, -0.1374, -0.0438, -0.0302, -0.0325, -0.0791,
         -0.0830, -0.1128,  0.1007, -0.0124,  0.0999, -0.0620, -0.0127, -0.0087,
      

In [25]:
dmc.fit(X_train, y_train)

/Users/dfrybrum/anaconda3/envs/debias/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
/Users/dfrybrum/anaconda3/envs/debias/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:167: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/dfrybrum/anaconda3/envs/debias/lib/python3

DebiasMClassifier(batch_str=16.666666666666668,
                  x_val=array([[  4, 696, 286, ..., 398, 240, 343],
       [  4, 513, 666, ..., 744, 472, 121],
       [  4, 896, 223, ..., 347,   4, 294],
       ...,
       [  4, 648, 845, ..., 791, 976, 578],
       [  4, 318, 960, ..., 640, 879, 992],
       [  4, 618, 300, ..., 912, 996, 621]]))